(data-loader-example)=
# Dataloader example

In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch

In [2]:
# cria um dataframe qualquer
df = pd.DataFrame({"id":[0,1,2,3,4],"text":["samara foi ate a feira",
                                              "junior nao saiu de casa",
                                              "cinthia comprou fruta",
                                              "leo correu atras do gato",
                                              "igor quebrou o dedo"]})

In [3]:
df.to_csv("data/dataset.csv",index=False)

In [4]:
df

,id,text
0,0,samara foi ate a feira
1,1,junior nao saiu de casa
2,2,cinthia comprou fruta
3,3,leo correu atras do gato
4,4,igor quebrou o dedo


In [5]:
# classe dataset

class samaraDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        text = self.data_frame["text"].iloc[idx]
        id_ = self.data_frame["id"].iloc[idx]
        sample = {"id":id_,'text': text}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [6]:
# lê e instancia o dataset
dataset = samaraDataset(csv_file='./data/dataset.csv',root_dir='./data/')
# cria o dataloader
dataloader = DataLoader(dataset, batch_size=2,
                        shuffle=True, num_workers=4)

In [7]:
# duas funções quaisquer pra simular o tokenizer e o cara que calcular os embeddings do bert
def bert_tokenizer(text):
    tokens = text.split(" ")
    return tokens

def bert_calculator(text):
    return [5*len(item) for item in text]

In [8]:
# cria um dataframe vazio onde os dados serão salvos
pd.DataFrame({"id":[],"text":[],"tokens":[],"embeddings":[]}).to_csv("data/transformed_data.csv",index=False)

In [9]:
# faz o cálculo por batch
for i_batch, sample_batched in enumerate(dataloader):
    idx,sample = sample_batched.items()
    x = pd.DataFrame({"id":idx[1].numpy(),"text":sample[1]})
    x["tokens"] = x.apply(lambda x: bert_tokenizer(x["text"]),axis=1)
    x["embeddings"] = x.apply(lambda x: bert_calculator(x["tokens"]),axis=1)
    x.to_csv("data/transformed_data.csv",header=None,index=False,mode="a")

In [10]:
# vê como ficou o dataframe final
pd.read_csv("data/transformed_data.csv")

,id,text,tokens,embeddings
0,3,leo correu atras do gato,"['leo', 'correu', 'atras', 'do', 'gato']","[15, 30, 25, 10, 20]"
1,1,junior nao saiu de casa,"['junior', 'nao', 'saiu', 'de', 'casa']","[30, 15, 20, 10, 20]"
2,4,igor quebrou o dedo,"['igor', 'quebrou', 'o', 'dedo']","[20, 35, 5, 20]"
3,2,cinthia comprou fruta,"['cinthia', 'comprou', 'fruta']","[35, 35, 25]"
4,0,samara foi ate a feira,"['samara', 'foi', 'ate', 'a', 'feira']","[30, 15, 15, 5, 25]"
